In [1]:
import os
import shutil
import requests

In [2]:
APIKEY = os.environ['AIRTABLE_APIKEY']
BASE = 'appfJRhs6ZsWF5OtU'
AIRTABLE = 'https://api.airtable.com/v0'
FILENAME = 'asset.mp4'
OUTDIR = 'm3-assets'

In [3]:
def fetch(table, rid=None):
    if rid is None:
        ret = requests.get(f'{AIRTABLE}/{BASE}/{table}?view=website', headers=dict(Authorization=f'Bearer {APIKEY}')).json()
        return [dict(**r['fields'], __id=r['id']) for r in ret['records']]
    else:
        ret = requests.get(f'{AIRTABLE}/{BASE}/{table}/{rid}', headers=dict(Authorization=f'Bearer {APIKEY}')).json()
        return ret['fields']

In [4]:
os.makedirs(OUTDIR, exist_ok=True)
asset_filename = os.path.join(OUTDIR, FILENAME)
if not os.path.exists(asset_filename):
    assets = fetch('Assets')
    asset = assets[0]['Attachments'][0]['url']
    with open(asset_filename, 'wb') as dst:
        src = requests.get(asset, stream=True).raw
        shutil.copyfileobj(src, dst)


In [5]:
from moviepy.video.io.VideoFileClip import VideoFileClip
a = VideoFileClip(asset_filename).audio
a.duration

2600.71

In [6]:
from moviepy.audio.AudioClip import CompositeAudioClip
import moviepy.audio.fx.all as afx
import moviepy.editor as editor


segments = fetch('Segment')
for segment in segments:
    out_filename = os.path.join(OUTDIR, f"{segment['title']}.mp3")
    timestamps = [fetch('Audio Editing', rid) for rid in segment['audio_editing']]
    timestamps = [t for t in timestamps if t['length']]
    clips = []
    for timestamp in timestamps:
        clip = a.copy()
        clip = clip.subclip(timestamp['start'], timestamp['end'])
        clip = clip.fx( afx.audio_fadein, 0.5)\
                   .fx( afx.audio_fadeout, 0.5)
        clips.append(clip)
    clip = editor.concatenate_audioclips(clips)
    clip.write_audiofile(out_filename)
    segment['duration'] = clip.duration
    print(segment['duration'])
    

chunk:  10%|▉         | 63/640 [00:00<00:00, 629.92it/s, now=None]

MoviePy - Writing audio in m3-assets/0: Introduction.mp3


MoviePy - Done.
29


chunk:  12%|█▏        | 103/883 [00:00<00:00, 1006.67it/s, now=None]

MoviePy - Writing audio in m3-assets/1: M.F - filling time.mp3


...

MoviePy - Writing audio in m3-assets/8: W.D - the GERD (dam) is a non consumptive project.mp3


MoviePy - Done.
19.5


chunk:   7%|▋         | 77/1103 [00:00<00:01, 768.17it/s, now=None]

MoviePy - Writing audio in m3-assets/9: M.M - sustainable water use.mp3


MoviePy - Done.
50


In [7]:
%%bash
git add m3-assets/
git status
git commit -m "Update M3 sound assets"
git push

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   m3-assets/4: W.D - why is it coming to a head now & Drought and climate.mp3
	modified:   m3-assets/6:M.M - Existential issue.mp3
	modified:   m3-assets/9: M.M - sustainable water use.mp3

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	.vscode/

[master fbceecf] Update M3 sound assets
 3 files changed, 0 insertions(+), 0 deletions(-)
 rewrite m3-assets/6:M.M - Existential issue.mp3 (79%)


To github.com:akariv/atlas-medliq.git
   9e9ae4a..fbceecf  master -> master


In [8]:
import time
cb = str(time.time())

for segment in segments:
    url = f'https://raw.githubusercontent.com/akariv/atlas-medliq/master/m3-assets/{segment["title"]}.mp3#{cb}'
    null_request = dict(
        records=[dict(
            id=segment['__id'],
            fields=dict(
                audio=[],
                duration=0
            )
        )]
    )
    full_request = dict(
        records=[dict(
            id=segment['__id'],
            fields=dict(
                audio=[dict(
                    url=url
                )],
                duration=float(segment['duration'])
            )
        )]
    )
    resp = requests.patch('https://api.airtable.com/v0/appfJRhs6ZsWF5OtU/Segment',
                           json=null_request, 
                           headers=dict(Authorization=f'Bearer {APIKEY}'))
    resp = requests.patch('https://api.airtable.com/v0/appfJRhs6ZsWF5OtU/Segment',
                           json=full_request, 
                           headers=dict(Authorization=f'Bearer {APIKEY}'))
    print(resp.json()['records'][0]['fields']['audio'][0])


{'id': 'attuDk4cnUKcMVs2M', 'url': 'https://raw.githubusercontent.com/akariv/atlas-medliq/master/m3-assets/0: Introduction.mp3#1611424781.821182', 'filename': '0:%20Introduction.mp3'}
{'id': 'atto42wazssllPLVK', 'url': 'https://raw.githubusercontent.com/akariv/atlas-medliq/master/m3-assets/1: M.F - filling time.mp3#1611424781.821182', 'filename': '1:%20M.F%20-%20filling%20time.mp3'}
{'id': 'attiAnC0qpmUUG4Dr', 'url': 'https://raw.githubusercontent.com/akariv/atlas-medliq/master/m3-assets/2: M.M - The real issue: Egypt control over the Nile & 11 countries.mp3#1611424781.821182', 'filename': '2:%20M.M%20-%20The%20real%20issue:%20Egypt%20control%20over%20the%20Nile%20&%2011%20countries.mp3'}
{'id': 'attBvwnzHcQV3TZlu', 'url': 'https://raw.githubusercontent.com/akariv/atlas-medliq/master/m3-assets/3: M.S - the role of Sudan & negotiations, not war.mp3#1611424781.821182', 'filename': '3:%20M.S%20-%20the%20role%20of%20Sudan%20&%20negotiations,%20not%20war.mp3'}
{'id': 'attzzhfMDcS9w5j2b', 'u